In [16]:
import os, glob
import subprocess
import tempfile
from tqdm import tqdm
import copy
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray
from rioxarray import merge
from osgeo import gdal

os.environ['GDAL_NUM_THREADS'] = 'ALL_CPUS'

In [17]:
rgi = 3
version = '62'
MAX_VAL_MAPBOX = 800
cmap = plt.get_cmap('turbo', 256)
PATH_TIFFS_IN = f"/media/maffe/nvme/iceboost_global_deploy/iceboost_20250212/RGI{version}/rgi{rgi}"
tif_filenames = [os.path.basename(file) for file in glob.glob(f'{PATH_TIFFS_IN}/*.tif')]
output_directory = f"/media/maffe/nvme/iceboost_global_deploy/iceboost_20250212/RGI62/mapbox_rgi{rgi}/"
print(f"We will process {len(tif_filenames)} files")

We will process 4556 files


In [18]:
# Process individual TIF files and save them as .tif in the temporary /tmp folder (we will remove them in the end)
# TODO: multi-cpu this for loop
processed_tifs = []

files = tif_filenames

for n, tif_name in tqdm(enumerate(files), total=len(files), leave=True):
    #print(n, '/', len(tif_filenames), tif_name)
    # open the tif and extract the ice thickness, which is the first variable
    tif = rioxarray.open_rasterio(f"{PATH_TIFFS_IN}/{tif_name}").sel(band=1) # EPSG:4326
    #tif.plot(cmap='turbo')
    #plt.show()
    tif = tif.squeeze().rio.reproject(dst_crs="EPSG:3857").clip(0, MAX_VAL_MAPBOX).fillna(0)
    rgba_data = cmap(tif.values / MAX_VAL_MAPBOX) * 255
    rgba_data[:, :, 3] = 127  # Set alpha to half transparency
    rgba_data[tif.values == 0] = 0
    rgba_data = rgba_data.astype(np.uint8)

    #fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5)
    #ax1.imshow(tif.values, cmap='gray')
    #ax2.imshow(rgba_data[:, :, 0], cmap='Reds')
    #ax3.imshow(rgba_data[:, :, 1], cmap='Greens')
    #ax4.imshow(rgba_data[:, :, 2], cmap='Blues')
    #ax5.imshow(rgba_data[:, :, 3], cmap='gray')
    #plt.show()

    rgb_data_array = xr.DataArray(
        rgba_data, dims=('y', 'x', 'band'),
        coords={'x': tif.coords['x'], 'y': tif.coords['y']}
    ).transpose('band', 'y', 'x')

    rgb_data_array.rio.write_crs("EPSG:3857", inplace=True)
    rgb_data_array.rio.write_nodata(0, inplace=True)

    # 1 pixel glaciers are skipped 
    if rgb_data_array.shape[1] == 1 or rgb_data_array.shape[2] == 1:
        continue

    #processed_tifs.append(rgb_data_array)

    # Save to a temporary file in the /tmp folder
    temp_file = tempfile.NamedTemporaryFile(suffix=".tif", delete=False).name
    rgb_data_array.rio.to_raster(temp_file, compress="deflate")
    processed_tifs.append(temp_file)

100%|███████████████████████████████████████| 4556/4556 [20:58<00:00,  3.62it/s]


In [19]:
vrt_path = "temp.vrt"  # Temporary VRT file

try:
    # Step 1. Virtual Raster (VRT)
    print(f'Begin VRT creation')
    t1_0 = time.time()
    gdal.BuildVRT(vrt_path, processed_tifs)
    if os.path.exists(vrt_path):
        print(f"{vrt_path} successfully created.")
        vrt_size = os.path.getsize(vrt_path)
        print(f"Size of {vrt_path}: {vrt_size / 1024:.2f} KB")
    else:
        raise FileNotFoundError(f"Failed to create {vrt_path}.")
    print(f"End VRT creation in {time.time()-t1_0}")

    # Step 2. Create a temporary GeoTIFF with the mask applied
    # gdal.Warp could be a slightly faster option than gdal.Translate
    with tempfile.NamedTemporaryFile(suffix=".tif", delete=False) as temp_tif:
        temp_tif_path = temp_tif.name
    translate_command = ['gdal_translate', '-mask', '4', '-co', 'COMPRESS=DEFLATE', vrt_path, temp_tif_path]

    print(f'Begin to translate command')
    t2_0 = time.time()
    subprocess.run(translate_command, check=True)
    file_size = os.path.getsize(temp_tif_path)  # File size in bytes
    file_size_mb = file_size / (1024 * 1024)  # Convert to MB
    print(f"Temporary masked GeoTIFF created at: {temp_tif_path} ({file_size_mb:.2f} MB) in {time.time()-t2_0:.2f} seconds")

    # Step 3. Run gdal2tiles on the result of gdal_translate
    tiles_command = ['gdal2tiles.py',
                     '-z', '5-10',
                     '-x',  # Exclude transparent tiles from result tileset
                     '-r', 'bilinear',
                     temp_tif_path, output_directory]
    subprocess.run(tiles_command, check=True)
    print(f"gdal2tiles completed successfully. Tiles saved to: {output_directory}")

finally:
    # Step 4. Remove temporary files
    for file_path in processed_tifs + [temp_tif_path, vrt_path]:
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Deleted {file_path}.")

Begin VRT creation
temp.vrt successfully created.
Size of temp.vrt: 8230.72 KB
End VRT creation in 4.58088755607605
Begin to translate command
Input file size is 59174, 54285
0...10...20...30...40...50...60...70...80...90...100 - done.
Temporary masked GeoTIFF created at: /tmp/tmpz0236ujs.tif (298.72 MB) in 39.84 seconds


Generating Base Tiles:


0...10...20...30...40...50...60...70...80...90...

Generating Overview Tiles:


100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
gdal2tiles completed successfully. Tiles saved to: /media/maffe/nvme/iceboost_global_deploy/iceboost_20250212/RGI62/mapbox_rgi3/
Deleted /tmp/tmp0aw4tgyj.tif.
Deleted /tmp/tmpxumy4v8y.tif.
Deleted /tmp/tmp3aw1q9on.tif.
Deleted /tmp/tmp31_h3xhs.tif.
Deleted /tmp/tmpeoqz11ub.tif.
Deleted /tmp/tmpsllufxdp.tif.
Deleted /tmp/tmp4tdcd9o5.tif.
Deleted /tmp/tmpaj7ilp_a.tif.
Deleted /tmp/tmpb90lskoo.tif.
Deleted /tmp/tmpbrbmd89_.tif.
Deleted /tmp/tmp8i9nr2c6.tif.
Deleted /tmp/tmp_04of5_k.tif.
Deleted /tmp/tmpy8pubegb.tif.
Deleted /tmp/tmpuox9534f.tif.
Deleted /tmp/tmprcaaajnk.tif.
Deleted /tmp/tmpx7ztgor4.tif.
Deleted /tmp/tmposgo92qs.tif.
Deleted /tmp/tmp_enmlnbr.tif.
Deleted /tmp/tmp86ei7vue.tif.
Deleted /tmp/tmpsptyumvt.tif.
Deleted /tmp/tmpenfxfkzz.tif.
Deleted /tmp/tmp3v44pklf.tif.
Deleted /tmp/tmpckj1qojp.tif.
Deleted /tmp/tmpd0izb5qs.tif.
Deleted /tmp/tmpj0mt55hk.tif.
Deleted /tmp/tmpr4hj6wm3.tif.
Deleted /tmp/tmpbn

In [20]:
# If something is left in /tmp remove any .tif file

# Find all .tif files in /tmp
junk_tif_files = glob.glob("/tmp/*.tif")

# Calculate total size of the .tif files
total_size_mb = sum(os.path.getsize(f) for f in junk_tif_files if os.path.isfile(f)) / (1024 ** 2)
num_files = len(junk_tif_files)

# Delete each file
for file_path in junk_tif_files:
    os.remove(file_path)
    #print(f"Deleted {file_path}")

print(f"Deleted {num_files} .tif files, freeing {total_size_mb:.2f} MB.")

Deleted 0 .tif files, freeing 0.00 MB.
